Linear Operators in Undergraduate Physical Chemistry
====================================================

## Lesson Information
### Lesson Overview

One of the two initial formulations of quantum mechanics (QM) was concerned with solving the time-independent wave equation (TIWE), which for a single particle of mass $m$ subject to an external potential $V(x,y,z)$ is given by
$$\frac{\hbar}{2m}\nabla^2\Psi(x,y,z) + V(x,y,z)\cdot\Psi(x,y,z) = E\cdot\Psi(x,y,z)$$
for the _wavefunction_ $\Psi(x,y,z)$, which describes the particle's ``amplitude'' in three dimensional space and encodes all knowable information about the particle and its properties. The second-order partial differential equation above was immediately successful for describing the properties of simple 1-dimensional systems (e.g., the atomic structure of the Hydrogen atom), however it was soon generalized to a more fundamental formulation in terms of Hermitian operators and the Hilbert spaces spanned by their eigenstates. While this formulation provides deep physical insight into the properties of quantum mechanical systems under consideration and indeed quantum mechanics itself, working with operators can be a significant conceptual leap for students in an undergraduate course in physical chemistry. 

The purpose of this lesson is therefore to help students apply the SymPy symbolic mathematics engine and its `algebra_with_sympy` interface to define and explore the properties of operators, as well as to apply them within the context of quantum mechanical model systems akin to those encountered in an undergraduate course in physical chemistry.

### Lesson Learning Objectives (LOs)
#### Cyberinfrastructure LOs
By the end of the lesson, students will be able to...
- Implement mathematical expressions for linear and nonlinear operators using SymPy,
- Construct matrix representations of linear operators in a given basis set using standard Python arrays,
- Perform array operations using the NumPy library, and
- Solve simple operator eigenvalue equations by matrix diagonalization using the NumPy library.

#### Content LOs
By the end of the lesson, students will be able to...
- Test and identify various operator properties, including linearity, Hermitian symmetry, and commutivity,
- Define operator eigenvalue equations and validate solutions, and
- Characterize the properties of operator eigenstates, including expectation value and variance of observables

### Lesson Prerequisites
#### Cyberinfrastructure Prereq's
- Familiarity with Python syntax (variable assignment, functions, control structures)
- Familiarity with the `algebra_with_sympy` library and applications to uni-/multivariate differential and integral calculus

#### Content Prereq's
This lesson focuses on both the exploration of operators and their properties with an application to quantum mechanical model systems, and the development of the _cyberinfrastructure skills_ necessary to do so. Students are expected to be passingly familiar with the following mathematical and physical chemistry concepts:
- Differential and integral calculus of scalar functions of one and/or multiple variables,
- The particle in an infinite square potential well as a quantum mechanical model system,
- Postulates of quantum mechanics, including in particular
    * Observables, measurement, and expectation value and variance; and
    * Uncertainty relationships, particularly the Heisenberg uncertainty principle.

:::{note} ℹ Functional Analysis and Dirac Notation
:icon:false
No student familiarity with Dirac notation or the rigorous mathematical formulation of quantum mechanics in terms of Hilbert spaces is assumed for this lesson.
:::

### Resources
- [MolSSI Workshop: Python Scripting for Computational Molecular Sciences](https://education.molssi.org/python_scripting_cms/)
- [MolSSI CMS Python Workshop: Introduction](https://education.molssi.org/python_scripting_cms/01-introduction/index.html)
- [Algebra with SymPy Documentation](https://gutow.github.io/Algebra_with_Sympy/algebra_with_sympy.html)
- [Demonstrations of `algebra_with_sympy` functionality with the `Equation` class](https://gutow.github.io/Algebra_with_Sympy/Demonstration%20of%20equation%20class.html)

### References
Portions of this lesson were inspired by the following texts, with their in-line abbreviations provided in parentheses:
- [OpenStax Calculus Volume 1](https://openstax.org/details/books/calculus-volume-1) (OSCv1)
- [OpenStax Calculus Volume 2](https://openstax.org/details/books/calculus-volume-2) (OSCv2)
- [OpenStax Calculus Volume 3](https://openstax.org/details/books/calculus-volume-3) (OSCv3)
- [McQuarrie & Simon, _Physical Chemistry: A Molecular Approach_, 2nd Ed.](https://chem.libretexts.org/Courses/Lock_Haven_University_of_Pennsylvania/CHEM321%3A_Quantum_Chemistry_and_Spectroscopy/TextMap_of_McQuarrie_and_Simon) (M&S)

## Notebook Setup
:::{warning} ⚒ Package Imports and Defining Helpful Functions, Constants, & Conversions
:icon: false

Execute the cells below to ensure all necessary Python libraries are loaded, and to define physical constants & conversion factors which may be useful when completing problems.
:::

In [ ]:
# EXECUTE: Import some packages that we will use later
from algebra_with_sympy import * # Automatically imports sympy
algwsym_config.output.solve_to_list = True # Makes automatic solution outputs a list for usability
import math
from math import log10
from sympy.plotting import plot as symplot
from IPython.display import display, Markdown
#from sympy.codegen.cfunctions import log10 # Makes common logarithm available as log10()
print("This notebook is running Algebra_with_Sympy version " + str(algwsym_version)+".")

In [ ]:
# EXECUTE: Define Python variables corresponding to fundamental constants & conversion factors for convenience
## Fundamental Physical Constants from CODATA 2022
c_val = 2.99792458E8               # Speed of light in a vacuum (m/s)
h_val = 6.62607015E-34             # Planck constant (J*s)
hbar_val = h_val / math.pi         # Angular Planck constant (J*s)
e_val = 1.602176634E-19            # Elementary charge (C)
epsilon_0_val = 8.854187818814E-12 # Vacuum electric permittivity (C**2 / J / m)
m_e_val = 9.109383713928E-31       # Electron rest mass (kg)
m_p_val = 1.6726219259552E-27      # Proton rest mass (kg)
R_H_val = 109680                   # Rydberg constant (1/cm)
k_B_val = 1.380649E-23             # Boltzmann constant (J/K)
N_A_val = 6.02214076E23            # Avogadro constant (1/mol)
gas_R_val = k_B_val * N_A_val      # Ideal gas constant (J/mol/K)

## Numerical conversion factors from CODATA 2022
eV2J = 1.602176634E-19             # Multiplicative conversion factor from eV to J

## I. General Operator Properties

In general, operators (denoted by a capital letter wearing a "hat", e.g., $\hat{\Omega}$) represent a set of instructions to transform a function, vector, or scalar (constant) number into something else by **"acting" to the right**:
$$\hat{\Omega}f = g$$
In this expression, we have made no assumptions about what any of $\hat{\Omega}$, $f$, or $g$ actually look like! Some possibilities include

| Operator Definition         | Action on constant $c$, variable $x$, or function $f(x)$ | English translation                              |
|-----------------------------|----------------------------------------------------------|--------------------------------------------------|
| $\hat{X} := x\cdot$         | $\hat{X}f(x) = x\cdot f(x)$                              | Multiply to the right by $x$                     |
| $\hat{D}_x := \frac{d}{dx}$ | $\hat{D}_x f(x) = \frac{d}{dx}f(x) = f'(x)$              | Differentiate $f(x)$ with respect to $x$         |
| ${\rm SQRT} := \sqrt{\;\;}$ | ${\rm SQRT} f(x) = \sqrt{f(x)}$                          | Take the square root of $f(x)$                   |

Because operators represent a set of instructions for transforming one object into another, let's begin by defining some scalars and functions using SymPy that we will use to explore operators and their properties below.

### I.A. Defining Trial Objects

:::{warning} ⚒ Defining Trial Functions & Variables
:icon: false
In the cells below, define the following:
- Python variable storing the scalar constant $c = 5$,
- Math variable $x\in\mathbb{R}$ declared as a SymPy variable,
- Python variable `f` storing the arbitrary scalar function $f(x)$,
- Python variable `g` storing the scalar function $g(x) = \cos{x}$
:::{note} 👀 Tip: Pretty Printing with Cell Return
:icon:false
To pretty-print a SymPy variable, expression, etc. with fancy math formatting, type the object's name on the final line of the cell and execute.
::::::

In [ ]:
# YOUR TURN: Define a Python variable `c` storing the scalar constant 5
c = 

## Print out your constant c using cell return
c

In [ ]:
# YOUR TURN: Define the real valued math variable x & print using cell return


In [ ]:
# YOUR TURN: Define a Python variable `f` representing arbitrary SymPy function f(x)


In [ ]:
# YOUR TURN: Define a Python variable `g` representing sin(x)


### I.B. Coding Operator Expressions

Unlike the mathematical notation $\hat{\Omega} f(x) = g(x)$ may suggest, operators do not necessarily only act by multiplying to the right. For example, while the operation $\hat{Z}g(x) = z\sin{x}$ does just involve multiplication, $\hat{D}_z g(x)$ does not:
\begin{equation}
\hat{D}_z g(x) = \frac{\partial}{\partial z} \sin{x} = 0
\end{equation}
Particularly when applying operators to mathematical objects using SymPy, it is important that the person coding the expression first translate the definition of the operator to its Python/SymPy representation, and only then perform that operation on the object of the operator's action. For the example above, you must actually evaluate the $z$-derivative of the function $g(x)$ using the SymPy `diff()` function:
```python
# Apply Dz = d/dz operator onto g(x) = sin(x):
Dz_g = diff(g,z)
```
In other words, whenever applying your operator would require calling a Python or SymPy function, _you must actually apply the operator yourself._
:::{hint}🐍 Operator Expressions Calling Python/SymPy Functions
:icon:false
When applying an operator to a SymPy object would require calling a Python or SymPy function, ***do not define a Python variable representing the operator!*** Instead, you must be the one to apply the operator by calling the appropriate functions on the object of the operator's action.

For example, for the operator $\hat{\Omega} = -\hat{D}_x^2 + \frac{1}{2}\hat{X}^2= -\frac{\partial^2}{\partial x^2} + \frac{1}{2}x^2$ and function $g(x) = \sin(x)$, the operator expression 
$$\hat{\Omega}g(x) = -\frac{\partial^2}{\partial x^2} \sin{x} + \frac{1}{2}x^2\sin{x}$$ 
could be translated into code and evaluated as
```python
var('x')
g = sin(x)
result = -diff(g,x,2) + 0.5*(x**2)*g
result
```
:::

In [ ]:
# EXECUTE: Execute the code for evaluating Omega*g by copying it from the Python hint above


:::{warning} ⚒ Your Turn: Evaluating Operator Expressions
:icon:false
Evaluate the following operator expressions involving real-valued functions in the cells below and pretty-print the result using a cell return.
| Part | Expression | Operator Definition |
|:----:|:-----------|:--------------------|
| (a)  | $\hat{\Omega} (x^3 - 2xy + y^2)$ | $\hat{\Omega} = {\rm SQRT} = \sqrt{\;\;}$ |
| (b)  | $\hat{\Omega} (Ay\sin{x} + Bx\cos{y})$ | $\hat{\Omega} = -\left(\frac{\partial}{\partial x} + \frac{\partial}{\partial y}\right)$; $A,B\in\mathbb{R}$ constants |
| (c)  | $\hat{\Omega} e^{-ax}$           | $\hat{\Omega} = \frac{d^3}{dx^3} + x^3$; $a\in\mathbb{R}$ constant |
| (d)  | $\hat{\Omega} (x^3 - 2x + 3)$    | $\hat{\Omega} = \int_0^1 dx$              |
| (e)  | $\hat{\Omega} (x^3y^2z^4)$       | $\hat{\Omega} = \nabla^2 = \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}$ |
:::

In [ ]:
# Your Turn: Part (a)


In [ ]:
# Your Turn: Part (b)


In [ ]:
# Your Turn: Part (c)


In [ ]:
# Your Turn: Part (d)


In [ ]:
# Your Turn: Part (e)


:::{tip} ✅ Check Yo'self 
:class:dropdown
:open:false
:icon:false
The results from evaluating the operator expressions above should be
| Part | Expression | Operator Definition | Result |
|:----:|:-----------|:--------------------|:-------|
| (a)  | $\hat{\Omega} (x^3 - 2xy + y^2)$ | $\hat{\Omega} = {\rm SQRT} = \sqrt{\;\;}$ | $\sqrt{x^{3} - 2 x y + y^{2}}$ |
| (b)  | $\hat{\Omega} (Ay\sin{x} + Bx\cos{y})$ | $\hat{\Omega} = -\left(\frac{\partial}{\partial x} + \frac{\partial}{\partial y}\right)$; $A,B\in\mathbb{R}$ constants | $- A y \cos{\left(x \right)} - A \sin{\left(x \right)} + B x \sin{\left(y \right)} - B \cos{\left(y \right)}$ |
| (c)  | $\hat{\Omega} e^{-ax}$           | $\hat{\Omega} = \frac{d^3}{dx^3} + x^3$; $a\in\mathbb{R}$ constant | $- a^{3} e^{- a x} + x^{3} e^{- a x}$ |
| (d)  | $\hat{\Omega} (x^3 - 2x + 3)$    | $\hat{\Omega} = \int_0^1 dx$ | $\frac{9}{4}$ | 
| (e)  | $\hat{\Omega} (x^3y^2z^4)$       | $\hat{\Omega} = \nabla^2 = \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}$ | $12 x^{3} y^{2} z^{2} + 2 x^{3} z^{4} + 6 x y^{2} z^{4}$ |
:::

### I.C. Operator Linearity
An operator is said to be _linear_ if it distributes into a sum in the normal way:
$$\hat{\Omega} \left(a f(x) + b g(x)\right) = a\hat{\Omega}f(x) + b\hat{\Omega}g(x),$$
or more generally,
$$\hat{\Omega} \left(\sum_{i} c_i f_i(x)\right) = \sum_i c_i\hat{\Omega}f_i(x).$$

:::{warning} ⚒ Your Turn: Assessing Operator Linearity
:icon:false
Determine if the operators below are linear by testing whether they distribute into the function $f(x) = 15x^3 - 2x^2 + 4x + 3$, i.e.,
$$\hat{\Omega}\left(15x^3 - 2x^2 + 4x + 3x^0\right) \stackrel{?}{=} 15\hat{\Omega}x^3 - 2\hat{\Omega}x^2 + 4\hat{\Omega}x + 3\hat{\Omega}x^0$$

| Part | Operator, $\hat{\Omega}$                                           |
|:----:|:-------------------------------------------------------------------|
| (a)  | $\hat{\Omega} = {\rm SQRT} = \sqrt{\;\;}$                          |
| (b)  | $\hat{\Omega} = \frac{d^3}{dx^3} + x^3$; $a\in\mathbb{R}$ constant |
| (c)  | $\hat{\Omega} = \int_0^1 dx$                                       |

:::{hint} 💡 Hint: Comparing SymPy Expressions
:icon:false
Assign the results from evaluating both sides of the expression above to Python variables, then compare with the `Expression1.equals(Expression2)` syntax:
```python
LHS = # Result from applying the operator to f(x) all at once on the LHS
RHS = # Result from distributing the operator into f(x) on the RHS
LHS.equals(RHS) # Will return `True` if Omega is linear, otherwise nonlinear!
```
Also, make sure you simplify your expressions with `simplify(Expression)` before comparing them! Otherwise, the two may actually be equal without SymPy realizing.
::::::

In [ ]:
# Your Turn: Part (a) Omega = SQRT()


In [ ]:
# Your Turn: Part (b) Omega = d^3/dx^3 + x^3


In [ ]:
# Your Turn: Part (b) Omega = int_0^1 dx


:::{tip} ✅ Check Yo'self 
:class:dropdown
:open:false
:icon:false
The results from evaluating the operator expressions above should be
| Part | Operator, $\hat{\Omega}$                  | Linear? |
|:----:|:------------------------------------------|:--------|
| (a)  | $\hat{\Omega} = {\rm SQRT} = \sqrt{\;\;}$ | No      |
| (b)  | $\hat{\Omega} = \frac{d^3}{dx^3} + x^3$   | Yes     |
| (c)  | $\hat{\Omega} = \int_0^1 dx$              | Yes     |
:::

### I.D. Products of Operators

Operators in a product are applied from the inside-out, in order of proximity to their object of operation:
$$\hat{\Omega}\hat{\Lambda} f(x) = \hat{\Omega}\left[\hat{\Lambda} f(x)\right] = \hat{\Omega} g(x) = h(x)$$

To evaluate expressions involving products of operators, it is essential to evaluate the result of each operation in the order it appears in the expression. For example, consider applying the operator $\hat{\Omega} = \frac{d^2}{dx^2} + kx^2$ for some constant $k\in\mathbb{R}$ twice in a row onto the real valued function $f(x) = Ne^{a x^2}$ for constants $a, N\in\mathbb{R}$:
\begin{align}
\hat{\Omega}\hat{\Omega}\left(Ne^{a x^2}\right) &= \hat{\Omega}\left[\hat{\Omega}\left(Ne^{a x^2}\right)\right]\\
&= \hat{\Omega}\left[\frac{d^2}{dx^2}Ne^{a x^2} + Nkx^2 e^{a x^2}\right]\\
&= \frac{d^2}{dx^2}\left[\frac{d^2}{dx^2}Ne^{a x^2} + Nkx^2 e^{a x^2}\right] + kx^2\left[\frac{d^2}{dx^2}Ne^{a x^2} + Nkx^2 e^{a x^2}\right]
\end{align}
To evaluate this expression in one line of code, you would need to _nest_ the application of the first operator within the applicaiton of the second:
```python
var('x k a N', real=True)
f = N*exp(a * x**2)
OOf = diff((diff(f,x,2) + k*x**2*f),x,2) + k*x**2*(diff(f,x,2) + k*x**2*f)
```
which is difficult both to read and to write. A much more straightforward approach is to compute the first operation $g(x) = \hat{\Omega}f(x)$ individually and then operate again on the result.

:::{hint}🐍 Evaluating Operator Products
:icon:false
The most straightforward way to evaluate the final result of an operator product is to compute each one individually before passing the intermediate result to the next operator in the product. For the example above, the result of the operator product $\hat{\Omega}\hat{\Omega}\left(Ne^{a x^2}\right)$ can be evaluated in SymPy as
```python
var('x k a N', real=True)
f = N*exp(a * x**2)
Of = diff(f,x,2) + (k * x**2)*f
OOf = diff(Of,x,2) + (k * x**2)*Of
```

In [ ]:
# Execute: Copy the code in the cell above and execute to evaluate Omega Omega f(x)


It is possible to construct an expression for the operator product itself, i.e., to form 
$$\hat{\Gamma} = \hat{\Omega}\hat{\Lambda}$$
by evaluating the action of the operator product on an arbitrary function $f(x)$.
:::{hint}🐍 Constructing the Expression for an Operator Product
:icon:false
The following will define an arbitrary SymPy function $f(x)$ and evaluate the operator product:
```python
var('x')
f = Function('f')(x)
Lf =                      # Apply operator Lambda to function f(x), return result
OLf =                     # Apply operator Omega to result of Lambda acting on f(x)
simplify(OOf).collect(f)  # Collect arbitrary function f(x) to facilitate mental refactor
```
where the final step of simplifying and collecting the function $f(x)$ will return an expression in terms of the function $f(x)$ which can be mentally refactored into an operator expression
$$\hat{\Omega}\hat{\Lambda}f(x) = \hat{\Gamma}f(x)$$
that may be implemented as normal.
:::

:::{warning} ⚒ Your Turn: Evaluating Operator Products Two Ways
:icon:false
Show that the operator expression $\hat{\Omega}\hat{\Omega}f(x)$ for $\hat{\Omega} = \frac{d^2}{dx^2} + kx^2$ and $f(x) = Ne^{a x^2}$ and constants $k,a,N\in\mathbb{R}$ produces identical results when evaluated either (a) by direct substitution or (b) by first constructing the expression for the operator product $\hat{\Gamma} = \hat{\Omega}\hat{\Lambda}$ and then evaluating the operation $\hat{\Gamma}f(x)$.
:::{hint} 💡 Hint: Compare Expressions from Both Approaches
:icon:false
Assign the expressions resulting from evaluating the operator product with both approaches to Python variables, then compare with the `Expression1.equals(Expression2)`. Also, make sure you simplify your expressions with `simplify(Expression)` before comparing them!
::::::

In [ ]:
# Direct substitution: OOf


In [ ]:
# Construct Lf w/ arbitrary function


In [ ]:
# Implement Lambda f(x) after mental refactoring, store as Lf


In [ ]:
# Compare results from direct substitution & formation of operator product 


### I.E. Operator Commutivity

In general, operators are not guaranteed to _commute_ with one another, i.e., 
$$\hat{\Omega}\hat{\Lambda} f(x) \neq \hat{\Lambda}\hat{\Omega} f(x).$$ 
It is possible to define the _commutator_ of two operators, $\left[\hat{\Omega},\hat{\Lambda}\right]$:
$$\left[\hat{\Omega},\hat{\Lambda}\right] = \hat{\Omega}\hat{\Lambda} - \hat{\Lambda}\hat{\Omega},$$
which can be evaluated for a particular function $f(x)$ as
$$\left[\hat{\Omega},\hat{\Lambda}\right]f(x) = \hat{\Omega}\hat{\Lambda}f(x) - \hat{\Lambda}\hat{\Omega}f(x).$$


:::{warning} ⚒ Your Turn: Evaluating Operator Commutators for a Particular Function
:icon:false
In the cells below , determine if the operators $\hat{\Omega}$ and $\hat{\Lambda}$ commute when applied to the real-valued functions provided by evaluating the commutator, and pretty-print the result using a cell return.
| Part | $\hat{\Omega}$ | $\hat{\Lambda}$ | Function, $f(x)$ |
|:----:|:--------------------------------------------------|:---------------------------|:-----------------|
| (a)  | $\hat{D}_x^2 = \frac{d^2}{dx^2}$                  | $\hat{D}_x = \frac{d}{dx}$ | $f(x) = A\sin{rx} + B\cos{sx}$; $A,B,r,s\in\mathbb{R}$ constant |
| (b)  | $\hat{X} = x\cdot$ (multiply to the right by $x$) | $\hat{D}_x = \frac{d}{dx}$ | $f(x) = e^{ax^2}$; $a\in\mathbb{R}$ constant |
:::

In [ ]:
# Your Turn: Part (a), O = Dx^2, L = Dx, f = exp(ax^2)
## Declare variables & define f(x)

## Evaluate OLf = Dx^2 Dx f

## Evaluate LOf = Dx Dx^2 f

## Evaluate [O,L]f = OLf - LOf

## Evaluate [O,L]: Simplify [O,L]f and divide out f


In [ ]:
# Your Turn: Part (b), O = X, L = Dx, f = Asin(rx) + Bcos(sx)


:::{tip} ✅ Check Yo'self 
:class:dropdown
:open:false
:icon:false
The results from evaluating the commutators above should be
| Part | $\hat{\Omega}$ | $\hat{\Lambda}$ | Function, $f(x)$ | Commutator Result, $\left[\hat{\Omega},\hat{\Lambda}\right]f$ |
|:----:|:---------------|:----------------|:-----------------|:--------------------------------------------------------------|
| (a)  | $\hat{D}_x^2 = \frac{d^2}{dx^2}$                  | $\hat{D}_x = \frac{d}{dx}$ | $f(x) = A\sin{rx} + B\cos{sx}$; $A,B,r,s\in\mathbb{R}$ constant | $\left[\hat{D}_x^2,\hat{D}_x\right]f(x) = 0$ |
| (b)  | $\hat{X} = x\cdot$ (multiply to the right by $x$) | $\hat{D}_x = \frac{d}{dx}$ | $f(x) = e^{ax^2}$; $a\in\mathbb{R}$ constant | $\left[\hat{X},\hat{D}_x\right]f(x) = 2 a x \left(x - 1\right)$ |
:::

To see if two operators commute in general, we must evaluate the commutator by applying it to an arbitrary function: 
$$\left[\hat{\Omega},\hat{\Lambda}\right]f = \hat{\Omega}\hat{\Lambda}f - \hat{\Lambda}\hat{\Omega}f.$$
Then, if the result is $\left[\hat{\Omega},\hat{\Lambda}\right]f = 0 = 0\cdot f$, it must be true in general that $\left[\hat{\Omega},\hat{\Lambda}\right] = 0$ which implies that the operators $\hat{\Omega}$ and $\hat{\Lambda}$ _always_ commute.

:::{warning} ⚒ Your Turn: Evaluating Operator Commutators for an Arbitrary Function
:icon:false
Determine if the operators $\hat{D}_x^2 = \frac{d^2}{dx^2}$ and $\hat{D}_x = \frac{d}{dx}$ commute in general by evaluating the commutator for an arbitrary real-valued function $f(x)$.
:::

In [ ]:
# Evaluate commutator [X,D_x] on random function f(x)


### I.F. Powers of Operators

An operator raised to a positive, whole-number power $n$ corresponds to the successive application of that operator $n$ times, i.e.,
$$\hat{O}^n f = \hat{O}\left[\hat{O}\left[\hat{O}\left[\ldots\left[\hat{O} f\right]\right]\right]\right].$$
In fact, this is the same way we typically write multiple derivatives in calculus. For example, writing the second derivative of the function $x^2$ like
$$\frac{d^2}{dx^2} x^2 = \frac{d}{dx}\left(\frac{d}{dx} x^2\right) = \frac{d}{dx} 2x = 2$$
is exactly the same as using the operator notation
$$\hat{D}_x^2 x^2 = \hat{D}_x\left[\hat{D}_x x^2\right] = \hat{D}_x 2x = 2$$

:::{warning} ⚒ Your Turn: Powers of Operators
:icon:false
Determine the expression for $\hat{\Omega}^2$ given the expressions for $\hat{\Omega}$ below, by evaluating the result of applying the operator product $\hat{\Omega}\hat{\Omega}$ to the arbitrary function $f(x)$. Then, evaluate $\hat{\Omega}^2 \left(e^{ax}\right)$ for constant $a\in\mathbb{R}$.
| Part | Operator, $\hat{\Omega}$                                                                     |
|:----:|:---------------------------------------------------------------------------------------------|
| (a)  | $\hat{\Omega} = \hat{D}_x^2 = \frac{d^2}{dx^2}$                                              |
| (b)  | $\hat{\Omega} = \hat{D}_x + \hat{X} = \frac{d}{dx} + x$                                      |
| (c)  | $\hat{\Omega} = \hat{D}_x^2 - 2\hat{X}\hat{D}_x + 1 = \frac{d^2}{dx^2} - 2x\frac{d}{dx} + 1$ |
:::

In [ ]:
# Your Turn: Part (a) Omega = Dx^2


In [ ]:
# Evaluate OO(e^ax) for (a)


In [ ]:
# Your Turn: Part (b) Omega = Dx + X


In [ ]:
# Evaluate OO(e^ax) for (b)


In [ ]:
# Your Turn: Part (c) Omega = Dx^2 - 2XDx + 1


In [ ]:
# Evaluate OO(e^ax) for (c)


:::{tip} ✅ Check Yo'self 
:class:dropdown
:open:false
:icon:false
The results from evaluating the commutators above should be
| Part | Operator, $\hat{\Omega}$                                                                     | $\hat{\Omega}^2$ Expression | Result of $\hat{\Omega}^2 \left(e^{ax}\right)$ |
|:----:|:---------------------------------------------------------------------------------------------|:----------------------------|:-----------------------------------------------|
| (a)  | $\hat{\Omega} = \hat{D}_x^2 = \frac{d^2}{dx^2}$                                              | $\frac{d^4}{dx^4}$          | $a^4 e^{ax}$ |
| (b)  | $\hat{\Omega} = \hat{D}_x + \hat{X} = \frac{d}{dx} + x$                                      | $2x\frac{d}{dx} + \left(x^2 + 1\right) + \frac{d^2}{dx^2}$ | $\left(a^2 + 2ax + x^2 + 1\right) e^{ax}$ |
| (c)  | $\hat{\Omega} = \hat{D}_x^2 - 2\hat{X}\hat{D}_x + 1 = \frac{d^2}{dx^2} - 2x\frac{d}{dx} + 1$ | $4x\frac{d}{dx} - 4x\frac{d^3}{dx^3} + \left(4x^2 - 4\right)\frac{d^2}{dx^2} + \frac{d^4}{dx^4} + 1$ | $a^4 e^{ax} - 4a^3 xe^{ax} + 4a^2 \left(x^2 - 1\right) e^{ax} + 4ax e^{ax} + 1$ |
:::

:::{danger} ☢ Warning: Operator Power vs. Power of Result
:icon:false
In general, $\hat{O}^n f \neq \left(\hat{O} f\right)^n$, which can be seen clearly with the example above:
$$\hat{D}_x^2 x^2 = 2 \neq x^2 = \left(\hat{D_x} x\right)^2$$
:::

## II. Operator Eigenvalue Problems

In general, the operator $\hat{\Omega}$) will act upon a function, vector, or scalar and transform it into something different, i.e.,
$$\hat{\Omega}f = g.$$
Certain operators, however, will have certain objects "of its own" for which
$$\hat{\Omega}\omega(x) = \alpha\omega(x),$$
where the object $\omega(x)$ is called an "eigenfunction," "eigenstate," or "eigenvector" of the operator $\hat{\Omega}$ and the constant $\alpha$ is called the "eigenvalue" associated with the eigenfunction $\omega(x)$. 

:::{note} ℹ "Eigen"functions & "Eigen"values
:icon:false
The German word "eigen" translates as "own," i.e., the eigenstates of an operator are the operator's own functions, and the eigenvalue associated with the eigenstate is that function's own value.
:::

### II.A. Identifying Operator Eigenstates

As it turns out, quantum mechanics almost always involves writing an eigenvalue equation given the form of an operator $\hat{\Omega}$ and then solving for its set of its eigenfunctions $\left\{\omega_n\right\}$ and their associated eigenvalues $\left\{\alpha_n\right\}$. This process is sufficiently involved that the details of solving eigenvalue problems in quantum mechanics will be left for a different lesson. Here, we will instead focus on determining if a given function _is_ an eigenfunction of a particular operator, as well as some general properties of operators and their eigenvalues.

:::{warning} ⚒ Your Turn: Identifying Operator Eigenstates
:icon:false
For each of the functions $f(x)$ below, determine if it is an eigenstate of the operator $\hat{\Omega}$ provided. If it is an eigenstate, determine its associated eigenvalue.
| Part | Operator, $\hat{\Omega}$                        | Function, $f(x)$                                                  |
|:----:|:------------------------------------------------|:------------------------------------------------------------------|
| (a)  | $\hat{\Omega} = \hat{D}^4_x = \frac{d^4}{dx^4}$ | $f(x) = e^{\alpha x}$; $\alpha\in\mathbb{R}$ constant |
| (b)  | $\hat{\Omega} = \hat{D}_y = \frac{d}{dy}$       | $f(x) = x^2 + 3x - 2$ |
| (c)  | $\hat{\Omega} = \hat{D}_y\hat{D_x} = \frac{d}{dy}\frac{d}{dx} = \frac{\partial^2}{\partial y\partial x}$ | $f(x,y) = y\cos{x}$ |
| (d)  | $\hat{\Omega} = -i\hbar\hat{D}_x = -i\hbar\frac{d}{dx}$; $\hbar\in\mathbb{R}$ constant | $f(x) = e^{ikx}$; $k\in\mathbb{R}$ constant |
| (e)  | $\hat{\Omega} = \hat{D}_x = \frac{d}{dx}$       | $f(x) = \sin{\pi x}$ |
| (f)  | $\hat{\Omega} = -\frac{\hbar^2}{2m}\frac{d^2}{dx^2}$; $\hbar\in\mathbb{R}$ constant | $f(x) = \sqrt{\frac{2}{\ell}}\sin{\frac{n\pi x}{\ell}}$; $\ell\in\mathbb{R}$, $n\in\mathbb{Z}^+$ constant |
:::

:::{hint} 💡 Hints
:icon:false
- To show that $f(x)$ is an eigenfunction of $\hat{\Omega}$, use `Equation.collect(term)`: 
    ```python
    f =           # define function
    Of =          # Apply operator Omega to function f 
    Of.collect(f) # will factor out original function from result to show it is an eigenfunction
    ```
- After showing $f(x)$ is an eigenfunction of $\hat{\Omega}$, divide operation result by $f(x)$ to isolate eigenvalue
    ```python
    f =                       # define function
    Of =                      # Apply operator Omega to function f 
    omega = Of.collect(f) / f # Isolate eigenvalue from operation result by dividing
    ```
:::

In [ ]:
# YOUR TURN: Part (a) Omega = Dx^4, f = exp(ax)
## Is f an eigenstate of Omega? If so, determine eigenvalue omega


In [ ]:
# YOUR TURN: Part (b) Omega = Dy = d/dy, f = x^2 + 3x - 2
## Is f an eigenstate of Omega? If so, determine eigenvalue omega


In [ ]:
# YOUR TURN: Part (b) Omega = DyDx = d^2/dydx, f = y*cos(x)
## Is f an eigenstate of Omega? If so, determine eigenvalue omega


In [ ]:
# YOUR TURN: Part (d) Omega = -i*hbar*Dx, f = exp(ikx)
## Is f an eigenstate of Omega? If so, determine eigenvalue omega


In [ ]:
# YOUR TURN: Part (e) Omega = Dx, f = sin(pi*x)
## Is f an eigenstate of Omega? If so, determine eigenvalue omega


In [ ]:
# YOUR TURN: Part (f) Omega = -(hbar^2/2m)Dx^2, f = sqrt(2/ell)sin((n*pi*x)/ell)
## Is f an eigenstate of Omega? If so, determine eigenvalue omega


:::{tip} ✅ Check Yo'self 
:class:dropdown
:open:false
:icon:false
The results for the parts above should be
| Part | Operator, $\hat{\Omega}$                        | Function, $f(x)$                                                  | Eigenfunction? | Eigenvalue, $\omega$ |
|:----:|:------------------------------------------------|:------------------------------------------------------------------|:--------------:|:---------------------|
| (a)  | $\hat{\Omega} = \hat{D}^4_x = \frac{d^4}{dx^4}$ | $f(x) = e^{\alpha x}$; $\alpha\in\mathbb{R}$ constant             | Yes            | $\omega = \alpha^4$  |
| (b)  | $\hat{\Omega} = \hat{D}_y = \frac{d}{dy}$       | $f(x) = x^2 + 3x - 2$                                             | Yes            | $\omega = 0$         |
| (c)  | $\hat{\Omega} = \hat{D}_y\hat{D_x} = \frac{d}{dy}\frac{d}{dx} = \frac{\partial^2}{\partial y\partial x}$ | $f(x,y) = y\cos{x}$ | ***NO!*** | N/A |
| (d)  | $\hat{\Omega} = -i\hbar\hat{D}_x = -i\hbar\frac{d}{dx}$; $\hbar\in\mathbb{R}$ constant | $f(x) = e^{ikx}$; $k\in\mathbb{R}$ constant | Yes | $\omega = \hbar k$ |
| (e)  | $\hat{\Omega} = \hat{D}_x = \frac{d}{dx}$       | $f(x) = \sin{\pi x}$                                              | ***NO!***      | N/A                  |
| (f)  | $\hat{\Omega} = -\frac{\hbar^2}{2m}\frac{d^2}{dx^2}$; $\hbar\in\mathbb{R}$ constant | $f(x) = \sqrt{\frac{2}{\ell}}\sin{\frac{n\pi x}{\ell}}$; $\ell\in\mathbb{R}$, $n\in\mathbb{Z}^+$ constant | Yes | $\omega = \frac{\hbar^2 \pi^2 n^2}{2m\ell^2}$ |
:::

### II.B. Operators with Simultaneous Eigenstates

In section I.E above, we saw that operators are often noncommutative, such that their commutator $\left[\hat{\Omega},\hat{\Lambda}\right]\neq 0$. From this property follows an interesting relationship when considering the eigenstates of operators who _do_ commute:
:::{note} ℹ️ Operator Eigenstates & Commutivity
:icon:false
If two linear operators $\hat{\Omega}$, $\hat{\Lambda}$ commute, i.e., $\left[\hat{\Omega},\hat{\Lambda}\right] = 0$, they will share a set of simultaneous eigenstates $\left\{\phi_n\right\}$ such that
\begin{align}
\hat{\Omega}\phi_n &= \omega_n\phi_n\\
\hat{\Lambda}\phi_n &= \lambda_n\phi_n = {\rm constant}\times\omega_n\phi_n,
\end{align}
and whose eigenvalues differ only up to a constant factor.
:::

:::{warning} ⚒ Your Turn: Operators with Simultaneous Eigenstates
:icon:false
Consider the operators $\hat{D}^2_x = \frac{\partial^2}{\partial x^2}$ and $\hat{D}^4_x = \frac{\partial^4}{\partial x^4}$. 
- (a) Show that these operators commute in general for an arbitrary function $F(x)$.
- (b) Show that the set of functions $f(x) = e^{ikx}$, where $k$ is a real constant, are eigenstates of both $\hat{D}^2_x$ and $\hat{D}^4_x$.
- (c) Show that the eigenvalues of $f(x)$ associated with the operators $\hat{D}^2_x$ and $\hat{D}^4_x$ differ only by a constant.
:::

In [ ]:
# YOUR TURN: (a) Show Dx2 and Dx4 commute


In [ ]:
# YOUR TURN: (b) Show f(x) = e^{ikx} is an eigenstate of both Dx2 & Dx4


In [ ]:
# YOUR TURN: (c) Show eigenvalues of f(x) w.r.t. Dx2 & Dx4 differ only up to a constant


:::{tip} ✅ Check Yo'self 
:class:dropdown
:open:false
:icon:false
By direct substitution, $f(x) = e^{ikx}$ should have been demonstrated as an eigenstate of both $\hat{\Omega} = \hat{D}^2_x = \frac{\partial^2}{\partial x^2}$ and $\hat{\Lambda} = \hat{D}^4_x = \frac{\partial^4}{\partial x^4}$, obeying the following eigenvalue equations:
\begin{align}
\hat{\Omega} f(x) = \hat{D}^2_x f(x) &= \omega_k f(x);\;\; \omega_k = -k^2\\
\hat{\Lambda} f(x) = \hat{D}^4_x f(x) &= \lambda_k f(x);\;\; \lambda_k = k^4\\
\end{align}
And therefore $\lambda_k = -k^2\omega_k = ({\rm constant})\times\omega_k$.  $\blacksquare$
:::

## III. Quantum Mechanical Operators

One postulate of the modern formulation of QM is that to each classical mechanical observable, there corresponds a linear Hermitian operator in quantum mechanics. 

### III.A. QM Operator Definitions
Several classical observables and their associated QM operators are provided in [](#tbl-qm-operators):

:::{table} Some classical mechanical observables and their associated linear Hermitian operators in quantum mechanics, adapted from Table 4.1 in M&S.
:label: tbl-qm-operators
:align: center
| Observable Name                           | Operator Definition  |
|:------------------------------------------|:--------------------:|
| Scalar position, $x$                      | $\hat{X} = x\cdot,\;\;\;\;{\rm multiply\ to\ the\ right\ by\ } x$ |
| Vector position, ${\bf r}$                | $\hat{R} = {\bf r}\cdot,\;\;\;\;{\rm vector\ dot\ product\ to\ the\ right\ by\ } {\bf r}$ |
| Linear momentum in $x$ direction, $p_x$   | $\hat{P}_x = -i\hbar\frac{\partial}{\partial x}$ |
| Total linear momentum, ${\bf P}$          | $\hat{\bf P} = -i\hbar\left({\bf i}\frac{\partial}{\partial x} + {\bf j}\frac{\partial}{\partial y} + {\bf k}\frac{\partial}{\partial z}\right)$ |
| Linear kinetic energy in $x$ direction, $T_x$    | $\hat{T}_x = -\frac{\hbar^2}{2m}\frac{\partial^2}{\partial x^2}$ |
| Total linear kinetic energy, $T$                 | $\hat{T} = -\frac{\hbar^2}{2m}\left(\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}\right) = -\frac{\hbar^2}{2m}\nabla^2$ |
| Angular momentum in $x$ direction, $L_x = yp_z - zp_y$  | $\hat{\mathcal{L}}_x = \hat{Y}\hat{\bf P}_z - \hat{Z}\hat{\bf P}_y$ |
| Angular momentum in $x$ direction, $L_y = zp_x - xp_z$  | $\hat{\mathcal{L}}_y = \hat{Z}\hat{\bf P}_x - \hat{X}\hat{\bf P}_z$ |
| Angular momentum in $x$ direction, $L_z = xp_y - yp_x$  | $\hat{\mathcal{L}}_z = \hat{X}\hat{\bf P}_y - \hat{Y}\hat{\bf P}_x$ |
| Total angular momentum squared, ${\bf L}^2 = {\bf L}\cdot{\bf L} = L_x^2 + L_y^2 + L_z^2$     | $\hat{\mathcal{L}}^2 = \hat{\mathcal{L}}_x^2 + \hat{\mathcal{L}}_y^2 + \hat{\mathcal{L}}_z^2$ |
| Total angular kinetic energy, $T = \frac{{\bf L}^2}{2I}$ | $\hat{T} = \frac{\hat{\mathcal{L}}^2}{2I}$ |
| Potential energy in $x$ direction, $V(x)$ | $\hat{V}_x = V(x)\cdot,\;\;\;\;{\rm multiply\ to\ the\ right\ by\ } V(x)$ |
| Total potential energy, $V(x,y,z)$        | $\hat{V} = V(x,y,z)\cdot,\;\;\;\;{\rm multiply\ to\ the\ right\ by\ } V(x,y,z)$ |
| Total energy, $E$                         | $\hat{H} = \hat{T} + \hat{V} = -\frac{\hbar^2}{2m}\nabla^2 + V(x,y,z)\cdot$ | 
:::

### III.B. Properties of QM Operators & Eigenstates

Beyond the general properties of linear operators we explored above, the additional criterion that all quantum mechanical operators be _Hermitian_ affords additional properties to these operators and their eigenstates that are worth exploring. While these properties are true for all of the Hermitian operators listed in [](#tbl-qm-operators), let us specifically consider a particle of mass $m$ subject to the potential energy 
$$V(x) = \begin{cases} 0 & 0\leq x\leq\ell\\ \infty & {\rm otherwise}\end{cases}$$
such that the particle is constrained to the region $0\leq x\leq\ell$. Inside this infinite square potential well, he Hamiltonian operator representing the total energy of the particle is given by
$$\hat{H} = -\frac{\hbar^2}{2m}\frac{d^2}{dx^2}$$
where $\hbar = \frac{h}{2\pi}$ is the angular Planck constant. This Hamiltonian defines the eigenvalue equation
$$\hat{H}\psi_n(x) = E_n\psi_n(x),$$
where the eigenstates $\psi_n(x)$ can be shown to be
$$\psi_n(x) = B\sin{\frac{n\pi x}{\ell}};\;\; n\in\mathbb{Z}^+,\ B\in\mathbb{R}$$
with associated energy eigenvalues
$$E_n = \frac{\hbar^2 n^2 \pi^2}{2m\ell^2}.$$

#### III.B.1 Eigenstate Normalizability
:::{note} ℹ Eigenstates of Hermitian Operators are Normalizable
:icon:false
The eigenstates $\left\{\omega_n\right\}$ of a Hermitian operator $\hat{\Omega}$ are _normalizable_, meaning that it is always possible to find a multiplicative constant $N^2$ such that the overlap integral
$$N^2\int_{\mathbb{R}^3} d^{3}{\bf r}\ \omega_n^{\ast}({\bf r})\omega_n({\bf r}) = 1,$$
where the asterisk (${}^{\ast}$) represents the complex conjugate. In other words, it is always possible to find a _normalization constant_ $N$ such that for $\psi_n(x) = N\cdot\omega_n(x)$,
$$1 = \int_{\mathbb{R}^3} d^{3}{\bf r}\ \psi_n^{\ast}({\bf r})\psi_n({\bf r}).$$
:::

:::{warning} ⚒ Your Turn: Normalizing Hamiltonian Eigenstates
:icon:false
Determine the value of the normalization constant $B$ for the eigenfunctions $\psi_n(x) = B\sin{\frac{n\pi x}{\ell}}$ of the Hamiltonian operator for the particle in a 1-dimensional infinite square well. Then re-define the eigenfunction by substituting in the value of the normalization constant, and prove that it has been normalized.
:::

In [ ]:
# YOUR TURN: Normalize psi_n(x) for 1dpib
## Determine value for normalization constant

## Re-define eigenstate w/ value of B, prove normalized


#### III.B.2 Eigenstate Orthonormality

:::{note} ℹ Eigenstates of Hermitian Operators are Orthonormal
:icon:false
The eigenstates $\left\{\omega_n({\bf r})\right\}$ of a Hermitian operator $\hat{\Omega}$ can be chosen to be mutually orthogonal, meaning that their overlap integral
$$\int_{\mathbb{R}^3} d^{3}{\bf r}\ \omega^{\ast}_m({\bf r})\omega_n({\bf r}) = 0\ {\rm for\ }m\neq n.$$
Combined with Property III.B.1 above that all eigenstates are normalizable, this means that the eigenstates of a Hermitian operator can be chosen to be _orthonormal_, such that the overlap integral above becomes 
$$\int_{\mathbb{R}^3} d^{3}{\bf r}\ \omega^{\ast}_m({\bf r})\omega_n({\bf r}) = \delta_{mn},$$
where $\delta_{mn}$ is called the _Kronecker delta function_, defined as
$$\delta_{mn} = \begin{cases}0 & m\neq n\\ 1 & m=n\end{cases}$$
:::

:::{warning} ⚒ Your Turn: Hamiltonian Eigenstates are Orthonormal
:icon:false
Show that the set of normalized eigenstates $\psi_n(x)$ for the particle in the one-dimensional infinite square well are orthonormal.
:::{hint}💡 Hint: Cases
:icon:false
Define two eigenstates with different quantum numbers $\psi_r(x)$ and $\psi_s(x)$, and let SymPy handle the cases.
::::::

In [ ]:
# YOUR TURN: Show <psi_r|psi_s> = d_mn


#### III.B.3 Eigenvalue Realness

:::{note} ℹ Hermitian Operators Have Real Eigenvalues
:icon:false
If an operator $\hat{\Omega}$ is Hermitian, then all of the eigenvalues $\left\{\omega_n\right\}$ associated with its eigenstates $\left\{\omega_n({\bf r})\right\}$ are guaranteed to be real, i.e., 
$$\omega_n\in\mathbb{R}\ {\rm for\ all\ } \omega_n\in \left\{\omega_n\right\}.$$
:::

Thanks to the postulate mentioned above that all QM operators are Hermitian, this property guarantees that all observables associated with these operators are measurable, real numbers--even for operators and who have purely imaginary eigenstates!

:::{warning} ⚒ Your Turn: Measuring Real Momentum from an Imaginary Eigenstate
:icon:false
The eigenstates for the $\hat{P}_x$ operator given in [](#tbl-qm-operators) are
\begin{align}
\psi_1(x) &= A_1e^{ikx}\\
\psi_2(x) &= A_2e^{-ikx},
\end{align}
where the exponential prefactors $A_i$ and phase factor $k$ are real constants. In the cells below, 
- (a) Show by direct substitution that these functions are eigenstates of the $\hat{P}_x$ operator, and determine their eigenvalues.
- (b) Show that the eigenvalues are real by calling the `eigenvalue.is_real` method; this should return True if the expression `eigenvalue` is real.
:::

In [ ]:
# YOUR TURN: Show eigenfunctions of Px, determine eigenvalues & show real
## Define eigenstates


In [ ]:
# (a) Show psi_1 eigenstate, determine eigenvalue


In [ ]:
# (a) Show psi_2 eigenstate, determine eigenvalue


In [ ]:
# (b) Show eigenvalues p1 & p2 are real


#### III.B.4 Degenerate Eigenstates

:::{note} ℹ Eigenstate Degeneracy
:icon:false
Two eigenstates $\phi_1$, $\phi_2$ of a linear operator $\hat{\Omega}$ are said to be _degenerate_ if they share an eigenvalue, i.e., 
\begin{align}
\hat{\Omega}\phi_1 &= \omega\phi_1\\
\hat{\Omega}\phi_2 &= \omega\phi_2
\end{align}
Every linear combination of these degenerate eigenstates, e.g. $c_1\phi_1 + c_2\phi_2$, will also be an eigenstate of the operator $\hat{\Omega}$, sharing the same eigenvalue $\omega$ as its constituent eigenstates:
$$\hat{\Omega}\left(c_1\phi_1 + c_2\phi_2\right) = c_1\hat{\Omega}\phi_1 + c_2\hat{\Omega}\phi_2 = c_1\omega\phi_1 + c_2\omega\phi_2 = \omega\left(c_1\phi_1 + c_2\phi_2\right)$$
:::

:::{warning} ⚒ Your Turn: Forming Linear Combinations of Degenerate Eigenstates
:icon:false
The Hermitian operator $\hat{D}_{\phi}^2 = \frac{d^2}{d\phi^2}$ has two eigenstates
\begin{align}
\Phi_{+}(\phi) &= e^{+im\phi}\\
\Phi_{-}(\phi) &= e^{-im\phi}
\end{align}
where $\phi$ is a real-valued variable and $m$ is a real constant. In the cells below, 
- (a) Show by direct substitution that these eigenstates of $\hat{D}_{\phi}^2$ share an eigenvalue and are therefore degenerate, and
- (b) Show by direct substitution that the arbitrary linear combination $\Phi = c_1\Phi_{+} + c_2\Phi_{-}$ is also an eigenstate of $\hat{D}_{\phi}^2$, and that it is degenerate with $\Phi_{+}$ abd $\Phi_{-}$.
:::

In [ ]:
# YOUR TURN: (a) Show Phi_+ & Phi_- are degenerate eigenstates
## Variable declaration & eigenstate definition


In [ ]:
# (a) Eigenvalue of Phi_plus


In [ ]:
# (a) Eigenvalue of Phi_minus


In [ ]:
# (a) Phi_plus & Phi_minus degenerate?


In [ ]:
# (b) Showing linear combination degenerate


### III.C. Measurement

Now that we have developed the properties of operators in general and of linear Hermitian operators found in quantum mechanics, let's see how we can leverage this understanding to explore the physical properties of quantum mechanical systems themselves.

#### III.C.1 Expectation Value of an Observable

:::{note} ℹ Expectation Values of Observables & Operators
:icon:false
Another postulate of quantum mechanics states that if a system is described by a normalized wavefunction $\Psi(\tau)$, then a measurement of the classical observable $\omega$ corresponding to the QM operator $\hat{\Omega}$ will have the _expected value_ given by
$$\langle\ \omega\ \rangle = \langle\ \hat{\Omega}\ \rangle = \underset{\rm all\ space}{\int}d\tau\ \Psi^{\ast}(\tau)\hat{\Omega}\Psi(\tau),$$
where $\tau$ represents the collective variables of all particles within the QM system and the integration is performed over all possible values of this collective variable (i.e, "all space").
:::

:::{warning} ⚒ Your Turn: Expected Value of Position & Momentum for the Particle in a 1D Infinite Square Well
:icon:false
Determine the expected values of position and momentum for a particle of mass $m$ confined to the infinite square well $0\leq x\leq\ell$, assuming the particle is described by one of the eigenstates of the Hamiltonian operator that you normalized in Section III.B.1 above.
:::

In [ ]:
# YOUR TURN: <x> & <p_x> for particle in energy eigenstate for 1D infinite square well [0,ell]


:::{tip} ✅ Check Yo'self 
:class:dropdown
:open:false
:icon:false

For a particle in an energy eigenstate,
- $\langle\ \hat{X}\ \rangle = \frac{\ell}{2}$
- $\langle\ \hat{P}_x\ \rangle = 0$
:::

:::{note} ℹ Expected Value of an Observable for an Eigenstate of the Corresponding QM Operator
:icon:false
If the system is in an eigenstate $\psi_n(\tau)$ of the operator $\hat{\Omega}$, then the expected value of the classical observable $\omega$ associated with $\hat{\Omega}$ is
$$\langle\ \omega\ \rangle = \underset{\rm all\ space}{\int}d\tau\ \psi_n^{\ast}(\tau)\hat{\Omega}\psi_n(\tau) = \underset{\rm all\ space}{\int}d\tau\ \psi_n^{\ast}(\tau)\omega_n\psi_n(\tau) = \omega_n\underset{\rm all\ space}{\int}d\tau\ \psi_n^{\ast}(\tau)\psi_n(\tau) = \omega_n\left(1\right) = \omega_n.$$
In other words, if a system is described by the eigenstate of an operator and we perform a measurement of the associated classical observable, we expect to measure the eigenvalue corresponding to the eigenstate describing the system.
:::

:::{warning} ⚒ Your Turn: Expected Value of Energy for an Energy Eigenstate
:icon:false
Determine the expected value for the energy of a particle of mass $m$ confined to the infinite square well $0\leq x\leq\ell$, assuming the particle is described by one of the eigenstates of the Hamiltonian operator that you normalized in Section III.B.1 above, and show that it is equal to the energy eigenvalue
$$E_n = \frac{\hbar^2 n^2 \pi^2}{2m\ell^2}.$$
:::

In [ ]:
# YOUR TURN: <E> for particle in energy eigenstate for 1D infinite square well [0,ell]


In [ ]:
## Does E_avg equal E_n?


:::{tip} ✅ Check Yo'self 
:class:dropdown
:open:false
:icon:false
For a particle in an energy eigenstate, $\langle\ E\ \rangle = \frac{\hbar^2\pi^2 n^2}{2m\ell^2} = E_n$
:::

#### III.C.2 Variance of an Observable

:::{note} ℹ Variance of Observables & Operators
:icon:false
The _variance_ in a classical observable $\omega$ corresponding to the QM operator $\hat{\Omega}$ is given by
$$\sigma^2_{\omega} = \langle\ \hat{\Omega}^2\ \rangle - \langle\ \hat{\Omega}\ \rangle^2.$$
:::

:::{warning} ⚒ Your Turn: Variance of Position & Momentum for the Particle in a 1D Infinite Square Well
:icon:false
Determine the variance in the position and momentum of a particle of mass $m$ confined to the infinite square well $0\leq x\leq\ell$, assuming the particle is described by one of the eigenstates of the Hamiltonian operator that you normalized in Section III.B.1 above, and show that this state obeys the Heisenberg uncertainty relationship 
$$\sigma_x\sigma_p\geq\frac{\hbar}{2}$$
:::{hint}🐍 Testing Inequalities in SymPy
:icon:false
It is possible to test the truth value of a mathematical inequality in SymPy using the binary operators `>`, `>=`, `<`, and `<=`, e.g., 
```python
expr1 > expr2
```
for expressions assigned to Python variables `expr1` and `expr2`. To fully evaluate the inequality to its truth value, you may need to first simplify and float-evaluate each expression with `simplify(expr).evalf()`!
:::
:::{hint} 💡 Hints
:icon:false
1. To fully evaluate an inequality to its truth value using a binary relation operator, , you may need to first simplify and float-evaluate each expression with `simplify(expr).evalf()`.
2. Your expressions for $\sigma_x$ and $\sigma_p$ will involve the quantum number $n$, and SymPy won't know how to handle an unassigned value for this parameter when comparing the left- and right-hand sides of the Heisenberg uncertainty relationship. To get around this limitation, substitute the value for $n$ minimizes the value of left-hand side of the Heisenberg uncertainty relationship. If the inequality holds for this value of $n$, it will hold for all values of $n$!
::::::

In [ ]:
# YOUR TURN: Variance in position & momentum for 1D PiB energy eigenstate
## Variable declarations

## Define energy eigenstates psi_n


In [ ]:
## Variance in Position, sigma^2_x = <X^2> - <X>^2
### <X> = < psi_n | X | psi_n >

### <X^2> = < psi_n | X^2 | psi_n >

### Variance in position


In [ ]:
## Variance in Momentum, sigma^2_px = <Px^2> - <Px>^2


:::{tip} ✅ Check Yo'self 
:class:dropdown
:open:false
:icon:false
For a particle in an energy eigenstate,
- Finding the variance in position:
    * $\langle\ \hat{X}\ \rangle = \frac{\ell}{2}$
    * $\langle\ \hat{X^2}\ \rangle = \frac{\ell^{2}}{3} - \frac{\ell^{2}}{2 \pi^{2} n^{2}}$
    * $\sigma^2_x = \frac{\ell^{2}}{12} - \frac{\ell^{2}}{2 \pi^{2} n^{2}}$
- Finding the variance in momentum:
    * $\langle\ \hat{P}_x\ \rangle = 0$
    * $\langle\ \hat{P^2}_x\ \rangle = \frac{\pi^{2} \hbar^{2} n^{2}}{\ell^{2}}$
    * $\sigma^2_{p_x} = \frac{\pi^{2} \hbar^{2} n^{2}}{\ell^{2}}$
:::

In [ ]:
## Show consistent w/ Heisenberg
### StDev of position & momentum

### LHS of Heisenberg

### RHS of Heisenberg

### Test Heisenberg


:::{note} ℹ Variance of an Observable for an Eigenstate of the Corresponding QM Operator
:icon:false
If the system is in an eigenstate $\psi_n(\tau)$ of the operator $\hat{\Omega}$, then the variance in a measurement of the classical observable $\omega$ is
\begin{align}
\sigma^2_{\omega} &= \langle\ \hat{\Omega}^2\ \rangle - \langle\ \hat{\Omega}\ \rangle^2\\
&= \underset{\rm all\ space}{\int}d\tau\ \psi_n^{\ast}(\tau)\hat{\Omega}^2\psi_n(\tau) - \left[\underset{\rm all\ space}{\int}d\tau\ \psi_n^{\ast}(\tau)\hat{\Omega}\psi_n(\tau)\right]^2\\
&= \omega_n\left[\underset{\rm all\ space}{\int}d\tau\ \psi_n^{\ast}(\tau)\hat{\Omega}\psi_n(\tau)\right] - \left\{\omega_n\left[\underset{\rm all\ space}{\int}d\tau\ \psi_n^{\ast}(\tau)\psi_n(\tau)\right]\right\}^2\\
&= \omega_n^2\left[\underset{\rm all\ space}{\int}d\tau\ \psi_n^{\ast}(\tau)\psi_n(\tau)\right] - \omega_n^2\left(1\right)^2\\
&= \omega_n^2\left(1\right)^2 - \omega_n^2\\
\therefore \sigma^2_{\omega} &= 0.
\end{align}
Therefore, if a quantum mechanical system is described by an operator eigenstate, a measurement of the corresponding classical observable is expected to return the eigenvalue belonging to that eigenstate with zero variance in the measurement.
:::

:::{warning} ⚒ Your Turn: Variance in the Expected Energy for a Hamiltonian Eigenstate
:icon:false
Show that the variance in the expected value of energy for a particle of mass $m$ confined to the infinite square well $0\leq x\leq\ell$, assuming the particle is described by one of the eigenstates of the Hamiltonian operator that you normalized in Section III.B.1 above, is zero.
:::

In [ ]:
# YOUR TURN: sigma^2_E for particle in energy eigenstate for 1D infinite square well [0,ell]


### III.D. Uncertainty Relationships

One of the most famous results from quantum mechanics is the so called _Heisenberg Uncertainty Principle_, which states that the position and momentum for a quantum mechanical particle cannot be known simultaneously to infinite precision, and instead obey the relationship
$$\sigma_x\sigma_{p_x} \geq \frac{\hbar}{2}$$
where $\sigma_x = \sqrt{\sigma^2_x}$ and $\sigma_{p_x} = \sqrt{\sigma^2_{p_x}}$ represent the _standard uncertainty_ (a.k.a. the _standard deviation_) in the particle's position and momentum, respectively. Though famous, this particular uncertainty relationship is not special; in fact, many such uncertainty relationships exist between classical observables of quantum mechanical systems. Most notable about these uncertainty relationships is the fact that _they are inherent to the formulation of quantum mechanics itself_, rather than resulting from inaccuracies in measurement or poorly performed experiments. So what about quantum mechanics gives rise to uncertainty relationships?

:::{note}ℹ️ Uncertainty Relationship Between Classical Observables for a QM System
:icon:false
For a quantum mechanical system described by state function $\Psi(\tau)$, the two classical observables $\lambda$, $\omega$ corresponding to the linear Hermitian operators $\hat{\Lambda}$, $\hat{\Omega}$ will have the uncertainty relationship
$$\sigma_{\lambda}\sigma_{\omega} \geq \frac{1}{2}\left\vert\underset{\rm all\ space}{\int} d\tau\ \Psi^{\ast}(\tau)\left[\hat{\Lambda},\hat{\Omega}\right]\Psi(\tau)\right\vert,$$
where the vertical bars indicate the complelx modulus (a.k.a. absolute value) of the integral.
:::

:::{warning} ⚒ Your Turn: Deriving the Heisenberg Uncertainty Principle
:icon:false
Consider a particle of mass $m$ in an infinite square well $0\leq x\leq\ell$ described by a Hamiltonian eigenstate $\psi_n(x) = \sqrt{\frac{2}{\ell}}\sin{\frac{n\pi x}{\ell}}$. Use the definition of the uncertainty relationship between two classical observables given above to derive the Heisenberg Uncertainty Principle, 
$$\sigma_x\sigma_p \geq \frac{\hbar}{2},$$
and use your approach to argue that the Heisenberg Uncertainty is true in general, i.e., not just for particles in a 1-dimensional infinite square potential well described by an energy eigenstate.
:::{hint}💡 Hint
:icon:false
First derive the expression for the commutator $\left[\hat{X},\hat{P_x}\right]$ using the general function $F(x)$, then substitute into the integral and evaluate.
::::::

In [ ]:
# YOUR TURN: Derive the Heisenberg Uncertainty Principle
## Derive commutator [X,Px]


In [ ]:
## Substitute [X,Px] into integral on RHS of general uncertainty relationship


In [ ]:
## Argue for generality of result
